# Libraries

##  Remove warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Import libraries

In [ ]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Basic libraries
#
import time
import random
import pandas    as pd
import numpy     as np
from   tqdm      import tqdm


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Visualization library
#
import matplotlib.pyplot   as plt 


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Sklearn library
#
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#
# Torch libraries
#
import torch
import torch.nn                     as nn
import torch.nn.functional          as F
from   torch.utils.data             import DataLoader
from   torch.utils.data             import Dataset


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#
# User libraries
#
from utils.PerformanceMetrics import RegressionEvaluation
from utils.EarlyStopping      import *
from utils.LRScheduler        import *



from models.DeepTIMe import *

# Parameters

## CUDA

In [ ]:
try:
    gpus = tensorflow.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tensorflow.config.experimental.set_memory_growth(gpu, True)

            device = torch.device( 'cuda:0' ) 
            
            logical_gpus = tensorflow.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

        except RuntimeError as e:
            
            # Memory growth must be set before GPUs have been initialized
            print(e)
            
            device = torch.device( 'cpu' )
except:
    device = torch.device( 'cpu' )
    print('[INFO] Not GPU found - CPU selected')

## Neural networks parameters

In [ ]:
class Parameters():
    def __init__(self):
        self.description = 'DLinear model for time-series forecasting'
    
        # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        # Neural network model parameters
        #
        # Input sequence length - look-back
        self.Lag         = 3 * 24
        # Prediction sequence length
        self.Horizon     = 24
        #
        self.individual  = False
        self.enc_in      = 1
        self.kernel_size = 25
        
        # Training parameters
        #
        # Number of epochs
        self.epochs        = 1000
        # Batch size
        self.batch_size    = 32
        # Number of workers in DataLoader
        self.num_workers   = 0
        # Define verbose
        self.verbose       = True
        # Learning rate
        self.learning_rate = 1e-4
        # Trained model path
        self.model_path    = 'models/DLinear.pth'
        
        # Data handling
        #
        # Filename
        self.filename              = './data/Solar_Radiation_Torino.csv'
        # Target series name 
        self.targetSeries          = 'Torre Pellice Direct Shortwave Radiation'
        # Training-set percentage
        self.TrainingSetPercentage = 0.8
        # Data Log-transformation
        self.Transformation        = True
        # Scaling {'Standard', 'MinMax', 'Robust'}
        self.Scaling               = 'Standard'

args = Parameters()

# Data handling

## Import data


In [ ]:
# Start timer
#
start = time.time()

# Load data
#
df = pd.read_csv( args.filename )

print('[INFO] Data imported')
print('[INFO] Time: %.2f seconds' % (time.time() - start))

df.head(3)

In [ ]:
df = df[-10000:]

## Preprocess data

### Set index

In [ ]:
# Convert Date to 'datetime64'
#
df['Date'] = df['Date'].astype('datetime64')

# Set index
#
df.set_index('Date', inplace=True)


# Keep only selected time-series
#
df = pd.DataFrame( df[ [ args.targetSeries ] ] )


df.head( 3 )

### Split Training/Testing

In [ ]:
idx = int( df.shape[0] * args.TrainingSetPercentage )

df_train = df[ :idx ].dropna()
df_test  = df[ idx: ].dropna()

### Visualization

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(20, 3) )

df_train[ args.targetSeries ].plot(ax=ax, color='tab:blue' )
df_test[ args.targetSeries ].plot(ax=ax,  color='tab:orange')

plt.legend(['Training', 'Testing'], frameon = False, fontsize = 14)
plt.ylabel(args.targetSeries, size = 14)
plt.xlabel('Date', size = 14);
plt.xticks(size = 12);
plt.yticks(size = 12);

### Fixing Lag

In [ ]:
df_test = pd.concat([df_train.iloc[-args.Lag:], df_test])

## Preprocessing

## Data Transformation

In [ ]:
if (args.Transformation == True):
    
    print('[INFO] Data transformation applied')
    
    VALUE = np.ceil( max(abs( -df.min().min() ), 1.0) )
    
    df_train = np.log( df_train + VALUE)
    df_test  = np.log( df_test  + VALUE)
    
else:
    print('[INFO] No data transformation applied.')  

In [ ]:
if (args.Scaling == 'MinMax'):
    print('[INFO] Scaling: MinMax')
    
    for feature in df.columns:
        if (feature ==  args.targetSeries ): continue
        print('Feature: ', feature)        
        # Set scaler
        #
        scaler = MinMaxScaler()
        
        df_train[feature] = scaler.fit_transform( df_train[ feature ].to_numpy().reshape(-1,1) )
        df_test[feature]  = scaler.transform( df_test[ feature ].to_numpy().reshape(-1,1) )

        
    # Scaling of Target Series
    #
    scaler = MinMaxScaler()
    df_train[ args.targetSeries ] = scaler.fit_transform( df_train[  args.targetSeries  ].to_numpy().reshape(-1,1) )
    df_test[ args.targetSeries ]  = scaler.transform( df_test[  args.targetSeries  ].to_numpy().reshape(-1,1) )
            
elif (args.Scaling == 'Robust'):
    print('[INFO] Scaling: Robust')
    
    for feature in df.columns:
        if (feature ==  args.targetSeries ): continue
        print('Feature: ', feature)        
        # Set scaler
        #
        scaler = RobustScaler()
        
        df_train[feature] = scaler.fit_transform( df_train[ feature ].to_numpy().reshape(-1,1) )
        df_test[feature]  = scaler.transform( df_test[ feature ].to_numpy().reshape(-1,1) )

        
    # Scaling of Target Series
    #
    scaler = RobustScaler()
    df_train[ args.targetSeries ] = scaler.fit_transform( df_train[  args.targetSeries  ].to_numpy().reshape(-1,1) )
    df_test[ args.targetSeries ]  = scaler.transform( df_test[  args.targetSeries  ].to_numpy().reshape(-1,1) )
        
elif (args.Scaling == 'Standard'):
    print('[INFO] Scaling: Standard')

    for feature in df.columns:
        if (feature ==  args.targetSeries ): continue
        print('Feature: ', feature)
        # Set scaler
        #
        scaler = StandardScaler()
        
        df_train[feature] = scaler.fit_transform( df_train[ feature ].to_numpy().reshape(-1,1) )
        df_test[feature]  = scaler.transform( df_test[ feature ].to_numpy().reshape(-1,1) )

        
    # Scaling of Target Series
    #
    scaler = StandardScaler()

    df_train[ args.targetSeries ] = scaler.fit_transform( df_train[  args.targetSeries  ].to_numpy().reshape(-1,1) )
    df_test[ args.targetSeries ]  = scaler.transform( df_test[  args.targetSeries  ].to_numpy().reshape(-1,1) )             
else:
    print('[WARNING] Unknown data scaling. Standar scaling was selected')   
    
    for feature in df.columns:
        if (feature ==  args.targetSeries ): continue
        print('Feature: ', feature)
        # Set scaler
        #
        scaler = StandardScaler()
        
        df_train[feature] = scaler.fit_transform( df_train[ feature ].to_numpy().reshape(-1,1) )
        df_test[feature]  = scaler.transform( df_test[ feature ].to_numpy().reshape(-1,1) )

        
    # Scaling of Target Series
    #
    scaler = StandardScaler()

    df_train[ args.targetSeries ] = scaler.fit_transform( df_train[  args.targetSeries  ].to_numpy().reshape(-1,1) )
    df_test[ args.targetSeries ]  = scaler.transform( df_test[  args.targetSeries  ].to_numpy().reshape(-1,1) )  

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional, List, Union

import numpy as np
import pandas as pd


class TimeFeature(ABC):
    """Abstract class for time features"""
    def __init__(self, normalise: bool, a: float, b: float):
        self.normalise = normalise
        self.a = a
        self.b = b

    @abstractmethod
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        ...

    @property
    @abstractmethod
    def _max_val(self) -> float:
        ...

    @property
    def max_val(self) -> float:
        return self._max_val if self.normalise else 1.0

    def scale(self, val: np.ndarray) -> np.ndarray:
        return val * (self.b - self.a) + self.a

    def process(self, val: np.ndarray) -> np.ndarray:
        features = self.scale(val / self.max_val)
        if self.normalise:
            return features
        return features.astype(int)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(normalise={self.normalise}, a={self.a}, b={self.b})"


class SecondOfMinute(TimeFeature):
    """Second of minute, unnormalised: [0, 59]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(idx.second)

    @property
    def _max_val(self):
        return 59.0


class MinuteOfHour(TimeFeature):
    """Minute of hour, unnormalised: [0, 59]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(idx.minute)

    @property
    def _max_val(self):
        return 59.0


class HourOfDay(TimeFeature):
    """Hour of day, unnormalised: [0, 23]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(idx.hour)

    @property
    def _max_val(self):
        return 23.0


class DayOfWeek(TimeFeature):
    """Hour of day, unnormalised: [0, 6]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(idx.dayofweek)

    @property
    def _max_val(self):
        return 6.0


class DayOfMonth(TimeFeature):
    """Day of month, unnormalised: [0, 30]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(idx.day - 1)

    @property
    def _max_val(self):
        return 30.0


class DayOfYear(TimeFeature):
    """Day of year, unnormalised: [0, 365]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(idx.dayofyear - 1)

    @property
    def _max_val(self):
        return 365.0


class WeekOfYear(TimeFeature):
    """Week of year, unnormalised: [0, 52]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(pd.Index(idx.isocalendar().week, dtype=int) - 1)

    @property
    def _max_val(self):
        return 52.0

class MonthOfYear(TimeFeature):
    """Month of year, unnormalised: [0, 11]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(idx.month - 1)

    @property
    def _max_val(self):
        return 11.0


class QuarterOfYear(TimeFeature):
    """Quarter of year, unnormalised: [0, 3]"""
    def __call__(self, idx: pd.DatetimeIndex) -> np.ndarray:
        return self.process(idx.quarter - 1)

    @property
    def _max_val(self):
        return 3.0


str_to_feat = {
    # dictionary mapping name to TimeFeature function
    'SecondOfMinute': SecondOfMinute,
    'MinuteOfHour': MinuteOfHour,
    'HourOfDay': HourOfDay,
    'DayOfWeek': DayOfWeek,
    'DayOfMonth': DayOfMonth,
    'DayOfYear': DayOfYear,
    'WeekOfYear': WeekOfYear,
    'MonthOfYear': MonthOfYear,
    'QuarterOfYear': QuarterOfYear,
}


freq_to_feats = {
    # dictionary mapping frequency to list of TimeFeature functions
    'q': [QuarterOfYear],
    'm': [QuarterOfYear, MonthOfYear],
    'w': [QuarterOfYear, MonthOfYear, WeekOfYear],
    'd': [QuarterOfYear, MonthOfYear, WeekOfYear, DayOfYear, DayOfMonth, DayOfWeek],
    'h': [QuarterOfYear, MonthOfYear, WeekOfYear, DayOfYear, DayOfMonth, DayOfWeek, HourOfDay],
    't': [QuarterOfYear, MonthOfYear, WeekOfYear, DayOfYear, DayOfMonth, DayOfWeek, HourOfDay, MinuteOfHour],
    's': [QuarterOfYear, MonthOfYear, WeekOfYear, DayOfYear, DayOfMonth, DayOfWeek, HourOfDay, MinuteOfHour, SecondOfMinute],
}


def get_time_features(dates: pd.DatetimeIndex, 
                      normalise: bool = False, 
                      a: Optional[float] = 0., 
                      b: Optional[float] = 1.,
                      features: Optional[Union[str, List[str]]] = None) -> np.ndarray:
    """
    Returns a numpy array of date/time features based on either frequency or directly specifying a list of features.
    :param dates: DatetimeIndex object of shape (time,)
    :param normalise: Whether to normalise feature between [a, b]. If not, return as an int in the original feature range.
    :param a: Lower bound of feature
    :param b: Upper bound of feature
    :param features: Frequency string used to obtain list of TimeFeatures, or directly a list of names of TimeFeatures
    :return: np array of date/time features of shape (time, n_feats)
    """
    if isinstance(features, list):
        assert all([feat in str_to_feat.keys() for feat in features]), \
            f"items in list should be one of {[*str_to_feat.keys()]}"
        features = [str_to_feat[feat] for feat in features]
    elif isinstance(features, str):
        assert features in freq_to_feats.keys(), \
            f"features should be one of {[*freq_to_feats.keys()]}"
        features = freq_to_feats[features]
    else:
        print('[ERROR]')
        raise ValueError(f"features should be a list or str, not a {type(features)}")

    features = [feat(normalise, a, b)(dates) for feat in features]

    if len(features) == 0:
        return np.empty((dates.shape[0], 0))
    return np.stack(features, axis=1)

In [ ]:
# str_to_feat = {
#     # dictionary mapping name to TimeFeature function
#     'SecondOfMinute': SecondOfMinute,
#     'MinuteOfHour': MinuteOfHour,
#     'HourOfDay': HourOfDay,
#     'DayOfWeek': DayOfWeek,
#     'DayOfMonth': DayOfMonth,
#     'DayOfYear': DayOfYear,
#     'WeekOfYear': WeekOfYear,
#     'MonthOfYear': MonthOfYear,
#     'QuarterOfYear': QuarterOfYear,
# }

## Create Training/Testing data

In [ ]:
def create_dataset(df = None, Lag = 1, Horizon = 1, targetSeries = None, overlap = 1):
    
    if (targetSeries is None):
        targetSeries = df.columns[-1]
    
    dataX, dataY, dataDate, dataXTime, dataYTime = [], [], [], [], []
    
    for i in tqdm( range(0, df.shape[0] + 1  - Lag - Horizon, overlap) ):
        
        dataX.append( df.to_numpy()[i:(i+Lag)] )        
        dataY.append( df[ targetSeries ].to_numpy()[i + Lag : i + Lag + Horizon] )
        dataDate.append( df.index[i + Lag : i + Lag + Horizon].tolist() )
        #
        dataXTime.append( get_time_features(dates    = df.index[i : i+Lag],  
                                            features = ['DayOfWeek', 'HourOfDay']))
        dataYTime.append( get_time_features(dates    = df.index[i + Lag : i + Lag + Horizon],  
                                            features = ['DayOfWeek', 'HourOfDay']))


    return ( np.array(dataX).astype(np.float32), 
             np.array(dataY).astype(np.float32), 
             np.array(dataDate),
             np.array(dataXTime).astype(np.float32), 
             np.array(dataYTime).astype(np.float32) )


In [ ]:
trainX, trainY, _, trainXTime, trainYTime      = create_dataset(df           = df_train, 
                                                                Lag          = args.Lag, 
                                                                Horizon      = args.Horizon, 
                                                                targetSeries = args.targetSeries,
                                                                overlap      = 1,)
                               

testX,  testY, testDate, testXTime, testYTime  = create_dataset(df           = df_test, 
                                                                Lag          = args.Lag, 
                                                                Horizon      = args.Horizon, 
                                                                targetSeries = args.targetSeries,
                                                                overlap      = 1,)


# Last 10% of the training data will be used for validation
#
idx = int(0.9 * trainX.shape[0])
validX, validY, validXTime, validYTime = trainX[ idx: ], trainY[ idx: ], trainXTime[ idx: ], trainYTime[ idx: ]
trainX, trainY, trainXTime, trainYTime = trainX[ :idx ], trainY[ :idx ], trainXTime[ :idx ], trainYTime[ :idx ]

print('Training data shape:   ', trainX.shape, trainY.shape)
print('Validation data shape: ', validX.shape, validY.shape)
print('Testing data shape:    ', testX.shape,  testY.shape)

In [ ]:
# # Reshaping
# #
# trainY = np.expand_dims(trainY, axis = -1)
# validY = np.expand_dims(validY, axis = -1)
# testY  = np.expand_dims(testY,  axis = -1)

In [ ]:
class Data( Dataset ):
    def __init__(self, X, Y, XTime, YTime):
        self.X    = X
        self.Y    = Y
        self.XTime = XTime
        self.YTime = YTime

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[ idx ], self.Y[ idx ], self.XTime[ idx ], self.YTime[ idx ]
    

    
# Create training and test dataloaders
#
train_ds = Data(trainX, trainY, trainXTime, trainYTime)
valid_ds = Data(validX, validY, validXTime, validYTime)
test_ds  = Data(testX,  testY,  testXTime,  testYTime)


# Prepare Data-Loaders
#
train_dl = DataLoader(train_ds, batch_size = args.batch_size, num_workers = args.num_workers)
valid_dl = DataLoader(valid_ds, batch_size = args.batch_size, num_workers = args.num_workers)
test_dl  = DataLoader(test_ds,  batch_size = args.batch_size, num_workers = args.num_workers)
#
print('[INFO] Data loaders were created')

# Forecasting model: DeepTIMe

## Setup model

In [ ]:
# Initialize Neural Network
# 
model = DeepTIMe(datetime_feats  = trainXTime.shape[-1], 
                 layer_size      = 64, 
                 inr_layers      = 32, 
                 n_fourier_feats = 10, 
                 scales          = [0.1])


model.to( device )


print( model )

## Training parameters

In [ ]:
# Specify loss function
#
criterion = nn.MSELoss()

# Specify loss function
#
optimizer = torch.optim.Adam(params = model.parameters(), 
                             lr     = args.learning_rate)




# Early stopping
#
early_stopping = EarlyStopping(patience  = 30,
                               min_delta = 1e-5)


# LR scheduler
#
scheduler = LRScheduler(optimizer = optimizer, 
                        patience  = 10, 
                        min_lr    = 1e-10, 
                        factor    = 0.5, 
                        verbose   = args.verbose)

## Training process

In [ ]:
# Store training and validation loss
Loss = {
         'Train': [], 
         'Valid':  []
       }



# Set number at how many iteration the training process (results) will be provided
#
batch_show = (train_dl.dataset.__len__() // args. batch_size // 5)



# Main loop - Training process
#
for epoch in range(1, args.epochs+1):

    # Start timer
    start = time.time()
    
    # Monitor training loss
    #
    train_loss = 0.0
    valid_loss  = 0.0    
    
    
    
    ###################
    # Train the model #
    ###################
    batch_idx = 0
    for data, target, XTime, YTime in train_dl:
        
        # Clear the gradients of all optimized variables
        #
        optimizer.zero_grad()
        
        # Forward pass: compute predicted outputs by passing inputs to the model
        #
        if (device.type == 'cpu'):
            data   = torch.tensor(data,   dtype=torch.float32)
            target = torch.tensor(target, dtype=torch.float32)
        else:
            data   = torch.tensor(data,   dtype=torch.float32).cuda()
            target = torch.tensor(target, dtype=torch.float32).cuda()

            
        outputs = model( data, XTime, YTime ).squeeze(-1)
        

        
        # Calculate the loss
        #
        loss = criterion(outputs, target)
        
        
        
        # Backward pass: compute gradient of the loss with respect to model parameters
        #
        loss.backward()
        
        
        
        # Perform a single optimization step (parameter update)
        #
        optimizer.step()
        
        
        
        # Update running training loss
        #
        train_loss += loss.item()*data.size(0)
               
        # Increase batch_idx
        #
        batch_idx  += 1
        
        
        # Info
        #
        if (args.verbose == True and batch_idx % batch_show == 0):
            print('> Epoch: {} [{:5.0f}/{} ({:.0f}%)]'.format(epoch, batch_idx * len(data), len(train_dl.dataset), 100. * batch_idx / len(train_dl)))        

           
        
    # Print avg training statistics 
    #
    train_loss = train_loss / train_dl.dataset.X.shape[0]

    
    
    
    
    with torch.no_grad():
        for data, target, XTime, YTime in valid_dl:

            # Forward pass: compute predicted outputs by passing inputs to the model
            #
            if (device.type == 'cpu'):
                data   = torch.tensor(data, dtype=torch.float32)
                target = torch.tensor(target, dtype=torch.float32)
            else:
                data   = torch.tensor(data, dtype=torch.float32).cuda()
                target = torch.tensor(target, dtype=torch.float32).cuda()


            outputs = model( data, XTime, YTime ).squeeze(-1)
        
          

            # Calculate the loss
            #
            loss = criterion(outputs, target)
                
            # update running training loss
            valid_loss += loss.item()*data.size(0)
              

    # Print avg training statistics 
    #
    valid_loss = valid_loss / test_dl.dataset.X.shape[0]






    # Stop timer
    #
    stop  = time.time()
    
    
    # Show training results
    #
    print('\n[INFO] Train Loss: {:.6f}\tValid Loss: {:.6f} \tTime: {:.2f}secs'.format(train_loss, valid_loss, stop-start), end=' ')

   
    

    # Update best model
    #
    if (epoch == 1):
        Best_score = valid_loss
        
        torch.save(model.state_dict(), args.model_path)
        print('(Model saved)\n')
    else:
        if (Best_score > valid_loss):
            Best_score = valid_loss
            
            torch.save(model.state_dict(), args.model_path)
            print('(Model saved)\n')
        else:
            print('\n')
     
    
    # Store train/val loss
    #
    Loss['Train'] += [ train_loss ]
    Loss['Valid'] += [ valid_loss ]
    

    
    
    
    # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    # Learning rate scheduler
    #
    scheduler( valid_loss )
    
    
    # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    # Early Stopping
    #
    if ( early_stopping( valid_loss ) ): break

## Load optimized model

In [ ]:
# Load best model
#
model.load_state_dict( torch.load( args.model_path ) );
model.eval();

print('[INFO] Model loaded')

## Evaluation

### Get predictions

In [ ]:
pred = None
with torch.no_grad():
    for data, target, XTime, YTime in tqdm( test_dl ):

        data   = torch.tensor(data,   dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32)

        if (pred is None):
            pred = model( data, XTime, YTime ).squeeze(-1).detach().numpy()
        else:
            pred = np.concatenate([ pred, model( data, XTime, YTime ).squeeze(-1).detach().numpy() ])

In [ ]:
# # Reshaping...
# #
# testY = testY.squeeze(-1)
# pred  = pred.squeeze(-1)

### Apply inverse scaling/transformation

In [ ]:
# Apply inverse scaling
#
for i in range( args.Horizon ):
    testY[:,  i] = scaler.inverse_transform( testY[:,  i].reshape(-1,1) ).squeeze(-1)
    pred[:, i]   = scaler.inverse_transform( pred[:, i].reshape(-1,1) ).squeeze(-1)


# Apply inverse transformation   
#
if (args.Transformation == True):
    testY = np.exp( testY ) - VALUE
    pred  = np.exp( pred )  - VALUE

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
print('[INFO] Feature: ', args.targetSeries)
print('------------------------------------------------')
Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

for i in range( args.Horizon ):

    Prices = pd.DataFrame([])        

    Prices[ args.targetSeries ] = testY[:,i]
    Prices[ 'Prediction'      ] = pred[:,i]


    # Evaluation
    #
    MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )

    # Store results
    #
    Performance_Foresting_Model['RMSE']    += [ RMSE    ]
    Performance_Foresting_Model['MAE']     += [ MAE     ]
    Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
    Performance_Foresting_Model['R2']      += [ R2      ]

    # Present results
    #
    print('Horizon: %2i MAE %5.2f RMSE %5.2f SMAPE: %5.2f R2: %.2f' % (i+1, MAE, RMSE, SMAPE, R2) )

### Residual examination

In [ ]:
# from scipy import stats
# from statsmodels.graphics.tsaplots import plot_acf

# for i in range( args.Horizon ):

#     # Get actual values and predicted
#     #
#     Prices = pd.DataFrame([])        

#     Prices[ args.targetSeries ] = testY[:,i]
#     Prices[ 'Prediction'      ] = pred[:,i]
                        
#     # Calculate the residuals
#     #
#     res = (Prices[ args.targetSeries ] - Prices['Prediction']).to_numpy()
    
    
    
#     # === Visualization ===
#     #
#     fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (15, 2) )

#     # Plot residual histogram
#     #
#     ax[0].hist(res, bins = 50)    
    
#     # Plot AutoCorrelation plot
#     #
#     plot_acf( res, ax=ax[1] )       
#     ax[1].set_ylim([-1.05, 1.05])

## Examples

In [ ]:
# # Apply inverse scaling on trainX
# #
# for i in range( trainX.shape[1] ):
#     testX[:,  i, 0] = scaler.inverse_transform( testX[:, i, 0].reshape(-1,1) ).squeeze(-1)


# # Apply inverse transformation   
# #
# if (Transformation == True):
#     testX = np.exp( testX ) - VALUE

In [ ]:
subplots = [331, 332, 333, 334, 335, 336,  337, 338, 339]
plt.figure( figsize = (20, 15) )

# Select random cases
RandomInstances = [random.randint(1, testY.shape[0]) for i in range(0, 9)]


for plot_id, i in enumerate(RandomInstances):

    plt.subplot(subplots[plot_id])
    plt.grid()
#     plot_scatter(range(0, Lag), testX[i,:,0], color='b')
    plt.plot(testDate[i], testY[i], color='g', marker = 'o', linewidth = 2)
    plt.plot(testDate[i], pred[i],  color='r', marker = 'o', linewidth = 2)

    plt.legend(['Actual values', 'Prediction'], frameon = False, fontsize = 12)
#     plt.ylim([0, 100])
    plt.xticks(rotation=45)
plt.show()